In [11]:
import sys
import os

# Add the project root directory to Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"Added {project_root} to Python path")

Added /Users/diego/Documents/maestriaUTEC/ciclo05/deeplearning/lab02 to Python path


In [ ]:
import mlflow
import numpy as np

mlflow.set_tracking_uri("http://127.0.0.1:5000") 

def train_and_eval_model(params):
    with mlflow.start_run():
        
        mlflow.log_params(params)
        mse = np.random.uniform(0.1, 0.5)   # métrica ficticia
        mae = np.random.uniform(0.1, 0.3)   # métrica ficticia
        
        # 4. Loggear métricas
        mlflow.log_metrics({
            "MSE": mse,
            "MAE": mae
        })
        
        print(f"[{params['dataset']} | horizon={params['output_len']} | trend={params['trend_method']}] "
              f"MSE={mse:.3f}, MAE={mae:.3f}")

In [3]:
params = {
    "dataset": "ETTh1",
    "input_len": 96,
    "output_len": 24,
    "trend_method": "regression",
    "d_model": 64,
    "n_layers": 2,
    "batch_size": 32,
    "learning_rate": 0.001,
    "epochs": 10
}

train_and_eval_model(params)

[ETTh1 | horizon=24 | trend=regression] MSE=0.395, MAE=0.203
🏃 View run bedecked-crane-252 at: http://127.0.0.1:5000/#/experiments/0/runs/e5c64ad8ab8844e88a02a07f3f0e3f75
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0


In [ ]:
import numpy as np
from micn import MHDecomp

X = np.array([10, 12, 14, 13, 15, 17, 18, 20])

decomp = MHDecomp(kernel_sizes=[2, 3])
Xt, Xs = decomp.forward(X)

print("Serie original:", X)
print("Tendencia Xt :", np.round(Xt, 2))
print("Estacionalidad Xs:", np.round(Xs, 2))


Serie original: [10 12 14 13 15 17 18 20]
Tendencia Xt : [11.5  11.5  12.5  13.25 14.   15.5  17.08 18.67]
Estacionalidad Xs: [-1.5   0.5   1.5  -0.25  1.    1.5   0.92  1.33]


In [ ]:
# Clear Python cache and reimport everything fresh
# import sys
# import os

# # Remove any cached modules
# modules_to_remove = [key for key in sys.modules.keys() if key.startswith('micn')]
# for module in modules_to_remove:
#     del sys.modules[module]

from micn import TrendRegression
import numpy as np

Xt = np.array([11.5, 11.5, 12.5, 13.25, 14.0, 15.5, 17.08, 18.67])

reg_mean = TrendRegression(method="mean", output_len=3)
Ytrend_mean = reg_mean.forward(Xt)
print("Predicción tendencia (mean):", np.round(Ytrend_mean, 2))

reg_lin = TrendRegression(method="regression", output_len=3)
Ytrend_reg = reg_lin.forward(Xt)
print("Predicción tendencia (regression):", np.round(Ytrend_reg, 2))

Predicción tendencia (mean): [14.25 14.25 14.25]
Predicción tendencia (regression): [18.96 20.   21.05]


In [22]:
from micn import MICLayers

Xs = np.array([-1.5, 0.5, 1.5, -0.25, 1.0, 1.5, 0.92, 1.33])

mic = MICLayers(output_len=3, d_model=4, n_layers=1, scales=[2, 4])
Yseasonal = mic.forward(Xs)

print("Predicción estacionalidad:", np.round(Yseasonal, 2))


Predicción estacionalidad: [0.62 0.62 0.62]


In [ ]:
import numpy as np
from micn.model import MICNModel

X = np.array([10, 12, 14, 13, 15, 17, 18, 20])

model = MICNModel(input_len=8, output_len=3, trend_method="regression",
                  d_model=4, n_layers=1, scales=[2, 4])

Ypred, Ytrend, Yseasonal = model.forward(X)

print("Predicción total Ypred:", np.round(Ypred, 2))
print("Tendencia Ytrend:", np.round(Ytrend, 2))
print("Estacionalidad Yseasonal:", np.round(Yseasonal, 2))


Predicción total Ypred: [19.19 20.15 21.1 ]
Tendencia Ytrend: [18.41 19.36 20.32]
Estacionalidad Yseasonal: [0.78 0.78 0.78]


In [ ]:
from utils.windowing import create_windows

serie = [10, 11, 13, 12, 14, 13, 16, 18]

X, Y = create_windows(serie, input_len=4, output_len=2)

print("X:\n", X)
print("Y:\n", Y)


X:
 [[10 11 13 12]
 [11 13 12 14]
 [13 12 14 13]]
Y:
 [[14 13]
 [13 16]
 [16 18]]


In [ ]:
import sys
import os

modules_to_remove = [key for key in sys.modules.keys() if key.startswith('utils')]
for module in modules_to_remove:
    del sys.modules[module]

from utils import create_windows, split_data
import pandas as pd

df = pd.read_csv("../data/ETTm1.csv")

serie = df["OT"].values

X, Y = create_windows(serie, input_len=96, output_len=24)

(X_train, Y_train), (X_val, Y_val), (X_test, Y_test) = split_data(X, Y)

print("Tamaño total:", len(X))
print("Train:", len(X_train), "Val:", len(X_val), "Test:", len(X_test))

Tamaño total: 69561
Train: 48692 Val: 6956 Test: 13913
